In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 12
num_filter = 64
compression = 0.5
dropout_rate = 0.1

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

167993344/170498071 [============================>.] - ETA: 0s170500096/170498071 [==============================] - 42s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 64, dropout_rate = 0.1):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        #relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(BatchNorm)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 64, dropout_rate = 0.1):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 12)   2592        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 

__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, 8, 8, 12)     3312        activation_3[0][0]               
__________________________________________________________________________________________________
dropout_69 (Dropout)            (None, 8, 8, 12)     0           conv2d_70[0][0]                  
__________________________________________________________________________________________________
average_pooling2d_3 (AveragePoo (None, 4, 4, 12)     0           dropout_69[0][0]                 
__________________________________________________________________________________________________
batch_normalization_70 (BatchNo (None, 4, 4, 12)     48          average_pooling2d_3[0][0]        
__________________________________________________________________________________________________
conv2d_71 (Conv2D)              (None, 4, 4, 12)     1296        batch_normalization_70[0][0]     
__________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/36
50000/50000 [==============================] - 620s 12ms/step - loss: 1.3544 - acc: 0.5078 - val_loss: 1.2005 - val_acc: 0.5944
Epoch 2/36
19072/50000 [==========>...................] - ETA: 5:38 - loss: 0.9769 - acc: 0.652650000/50000 [==============================] - 588s 12ms/step - loss: 0.9085 - acc: 0.6783 - val_loss: 0.9055 - val_acc: 0.7000
Epoch 3/36
45184/50000 [==========================>...] - ETA: 52s - loss: 0.7563 - acc: 0.733450000/50000 [==============================] - 585s 12ms/step - loss: 0.7536 - acc: 0.7345 - val_loss: 0.8190 - val_acc: 0.7240
Epoch 4/36
50000/50000 [==============================] - 589s 12ms/step - loss: 0.6587 - acc: 0.7697 - val_loss: 0.7425 - val_acc: 0.7562
Epoch 5/36
 2944/50000 [>.............................] - ETA: 8:39 - loss: 0.5950 - acc: 0.793150000/50000 [==============================] - 590s 12ms/step - loss: 0.6004 - acc: 0.7897 - val_loss: 0.7409 - val_acc: 0.7485
E

In [13]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 50s 5ms/step
Test loss: 0.5772865676879883
Test accuracy: 0.8461


In [14]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files
files.download('DNST_model.h5')